In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python

#load packages
import sys #access to system parameters https://docs.python.org/3/library/sys.html
import pandas as pd #collection of functions for data processing and 
# analysis modeled after R dataframes with SQL like features
import matplotlib #collection of functions for scientific and publication-ready visualization
import numpy as np #foundational package for scientific computing
import scipy as sp #collection of functions for scientific computing and advance mathematics
# numpyより高度なことができる。フーリエ変換とかね。
import IPython
from IPython import display #pretty printing of dataframes in Jupyter notebook
# https://qiita.com/5t111111/items/7852e13ace6de288042f
# ipythonは便利なシェル？
import sklearn #collection of machine learning algorithms
# misc libraries
# mist = その他
import random
import time
# ignore warning
import warnings
warnings.filterwarnings('ignore')
# よくわからん原因不明の警告が表示されることがあるので、それを防止できる。

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
# Any results you write to the current directory are saved as output.

In [9]:
#Common Model Algorithms
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier

#Common Model Helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics

#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.tools.plotting import scatter_matrix

#Configure Visualization Defaults
#%matplotlib inline = show plots in Jupyter Notebook browser
%matplotlib inline
mpl.style.use('ggplot')  # なんかよくわからんが、プロットのスタイル
sns.set_style('white')  # 背景白、グリッドなし
pylab.rcParams['figure.figsize'] = 12,8  # matplotの調整

In [10]:
# nullを見に行く
# print('-'*12, 'Train:\n', data1.isnull())  # 中身がnullか調べて、nullならtrue 
print('Train column has with null values:\n', data1.isnull().sum())
# .sum()で、各columsごとに、Trueの数を数えてる
print('-'*10)
print('Test column has with null values:\n', data_val.isnull().sum())
# .sum()で、各columsごとに、Trueの数を数えてる
print('-'*10)


NameError: name 'data1' is not defined

In [11]:
#import data from file: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html
data_raw = pd.read_csv('../input/train.csv')
print('--------train data---------')
print(data_raw)
#a dataset should be broken into 3 splits: train, test, and (final) validation
#the test file provided is the validation file for competition submission
#we will split the train set into train and test data in future sections
data_val = pd.read_csv('../input/test.csv')
print('--------test data---------')
print(data_val)
#to play with our data we'll create a copy
#remember python assignment or equal passes by reference vs values,
# so we use the copy function: 
# https://stackoverflow.com/questions/46327494/python-pandas-dataframe-copydeep-false-vs-copydeep-true-vs
data1 = data_raw.copy(deep = True)
# http://kurochan-note.hatenablog.jp/entry/20110316/1300267023
print('--------data1---------')
print(data1)

#however passing by reference is convenient, because we can clean both datasets at once
data_cleaner = [data1, data_val]
# 一度にデータがそうさできて楽なんで、まとめる。
print('--------data_cleaner---------')
print(data_cleaner)

#preview data
print('---------raw info----------')
print (data_raw.info()) 
print('---------samples----------')
data_raw.sample(10) #https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sample.html

--------train data---------
     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
5              6         0       3   
6              7         0       1   
7              8         0       3   
8              9         1       3   
9             10         1       2   
10            11         1       3   
11            12         1       1   
12            13         0       3   
13            14         0       3   
14            15         0       3   
15            16         1       2   
16            17         0       3   
17            18         1       2   
18            19         0       3   
19            20         1       3   
20            21         0       2   
21            22         1       2   
22            23         1       3   
23            24         1       1   
24            25      

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
496,497,1,1,"Eustis, Miss. Elizabeth Mussey",female,54.0,1,0,36947,78.2667,D20,C
545,546,0,1,"Nicholson, Mr. Arthur Ernest",male,64.0,0,0,693,26.0000,NaN,S
563,564,0,3,"Simmons, Mr. John",male,NaN,0,0,SOTON/OQ 392082,8.0500,NaN,S
808,809,0,2,"Meyer, Mr. August",male,39.0,0,0,248723,13.0000,NaN,S
597,598,0,3,"Johnson, Mr. Alfred",male,49.0,0,0,LINE,0.0000,NaN,S
847,848,0,3,"Markoff, Mr. Marin",male,35.0,0,0,349213,7.8958,NaN,C
669,670,1,1,"Taylor, Mrs. Elmer Zebley (Juliet Cummins Wright)",female,NaN,1,0,19996,52.0000,C126,S
480,481,0,3,"Goodwin, Master. Harold Victor",male,9.0,5,2,CA 2144,46.9000,NaN,S
729,730,0,3,"Ilmakangas, Miss. Pieta Sofia",female,25.0,1,0,STON/O2. 3101271,7.9250,NaN,S
176,177,0,3,"Lefebre, Master. Henry Forbes",male,NaN,3,1,4133,25.4667,NaN,S


In [12]:
# age, cabin, embarkedの欠損を埋める
# ageは数値データ、cabinはcategorical
# age, embarkedを埋める。cabinは消す.cabinは欠損が多すぎる。
# print(data1['Cabin'])
# print(data1['Embarked'])
for dataset in data_cleaner:
    dataset['Age'].fillna(dataset['Age'].median(), inplace =  True)
    # inplaceで元のオブジェクト自体が変更される。
    dataset['Embarked'].fillna(dataset['Embarked'].mode()[0], inplace = True)
    dataset['Fare'].fillna(dataset['Fare'].median(), inplace = True)
data_column = ['PassengerId', 'Cabin', 'Ticket']
data1.drop(data_column, axis=1,  inplace = True)
#dropはindexを指定して削除できる。axis=1だと列で削除



In [13]:
data_raw.describe(include = 'all')
# uniqueはデータの種類(bool)
# top：最頻値
# uniqueとかtopはboolにのみ作用できる。数値データとかは無理ぽい
# freque：最も一般的な値のでる回数

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Yrois, Miss. Henriette (""Mrs Harbeck"")",male,NaN,NaN,NaN,CA. 2343,NaN,G6,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


In [14]:
print(data1)

     Survived  Pclass                                               Name  \
0           0       3                            Braund, Mr. Owen Harris   
1           1       1  Cumings, Mrs. John Bradley (Florence Briggs Th...   
2           1       3                             Heikkinen, Miss. Laina   
3           1       1       Futrelle, Mrs. Jacques Heath (Lily May Peel)   
4           0       3                           Allen, Mr. William Henry   
5           0       3                                   Moran, Mr. James   
6           0       1                            McCarthy, Mr. Timothy J   
7           0       3                     Palsson, Master. Gosta Leonard   
8           1       3  Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)   
9           1       2                Nasser, Mrs. Nicholas (Adele Achem)   
10          1       3                    Sandstrom, Miss. Marguerite Rut   
11          1       1                           Bonnell, Miss. Elizabeth   
12          

In [15]:
data_raw.describe(include = 'all')


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Yrois, Miss. Henriette (""Mrs Harbeck"")",male,NaN,NaN,NaN,CA. 2343,NaN,G6,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


In [16]:
###CREATE: Feature Engineering for train and test/validation dataset
for dataset in data_cleaner:
    dataset['Familysize'] = dataset['SibSp'] + dataset['SibSp'] + 1
    dataset['IsAlone'] = 1 # initialize by 1
    dataset['IsAlone'].loc[dataset['Familysize'] > 1] = 0
    # loc[]なのに注意参照元.loc[指定したい条件]
    # loc[]で、条件に合うリストをとってこれる
    dataset['Title'] = dataset['Name'].str.split(",", expand=True)[1].str.split(".", expand=True)[0]
    # pandasはstr型のデータに当たるとき、.str＝strアクセを使うと各データに対して処理を適用できる
    # expand=Trueで複数の列に分割できる
    # qcut, cutは連続値を任意の境界で区分けして離散値にする。=ビニング処理をする
    # qcutはビンに含まれる要素数を等しくビニング処理
    # cutは最大値と最小値の間で値の間隔が等しくなるよう分割
    dataset['Farebin'] = pd.qcut(dataset['Fare'], 4)
    # astypeでpandasの中身をキャストできる
    dataset['Agebin'] = pd.cut(dataset['Age'].astype(int), 5)

In [17]:
  #cleanup rare title names

stat_min = 10
print(data1['Title'].value_counts())
title_name = (data1['Title'].value_counts() < stat_min)
# 各タイトルに対して、出現数をカウントしたリストが生成され、その各要素に対して、判定
# print(title_name)


 Mr              517
 Miss            182
 Mrs             125
 Master           40
 Dr                7
 Rev               6
 Mlle              2
 Major             2
 Col               2
 Jonkheer          1
 Lady              1
 Ms                1
 Sir               1
 Capt              1
 Mme               1
 the Countess      1
 Don               1
Name: Title, dtype: int64


In [18]:
print(data1['Title'].value_counts())

 Mr              517
 Miss            182
 Mrs             125
 Master           40
 Dr                7
 Rev               6
 Mlle              2
 Major             2
 Col               2
 Jonkheer          1
 Lady              1
 Ms                1
 Sir               1
 Capt              1
 Mme               1
 the Countess      1
 Don               1
Name: Title, dtype: int64


In [19]:
# rareなタイトルなら、その他=Miscとする。
data1['Title'] = data1['Title'].apply(lambda x: 'Misc' if title_name.loc[x] == True else x)
# apply()各列に関数を適用

In [20]:
data1.info()
# 「有効データ数」「データ型」「メモリ使用量」などの総合的な情報を表示
# なんかよくわからんがobjectはstrでcategoryは値の範囲という理解にしとこう。無理。

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
Survived      891 non-null int64
Pclass        891 non-null int64
Name          891 non-null object
Sex           891 non-null object
Age           891 non-null float64
SibSp         891 non-null int64
Parch         891 non-null int64
Fare          891 non-null float64
Embarked      891 non-null object
Familysize    891 non-null int64
IsAlone       891 non-null int64
Title         891 non-null object
Farebin       891 non-null category
Agebin        891 non-null category
dtypes: category(2), float64(2), int64(6), object(4)
memory usage: 85.5+ KB


In [21]:
data1.sample(10)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Familysize,IsAlone,Title,Farebin,Agebin
860,0,3,"Hansen, Mr. Claus Peter",male,41.0,2,0,14.1083,S,5,0,Mr,"(7.91, 14.454]","(32.0, 48.0]"
681,1,1,"Hassab, Mr. Hammad",male,27.0,0,0,76.7292,C,1,1,Mr,"(31.0, 512.329]","(16.0, 32.0]"
266,0,3,"Panula, Mr. Ernesti Arvid",male,16.0,4,1,39.6875,S,9,0,Mr,"(31.0, 512.329]","(-0.08, 16.0]"
770,0,3,"Lievens, Mr. Rene Aime",male,24.0,0,0,9.5000,S,1,1,Mr,"(7.91, 14.454]","(16.0, 32.0]"
673,1,2,"Wilhelms, Mr. Charles",male,31.0,0,0,13.0000,S,1,1,Mr,"(7.91, 14.454]","(16.0, 32.0]"
147,0,3,"Ford, Miss. Robina Maggie ""Ruby""",female,9.0,2,2,34.3750,S,5,0,Miss,"(31.0, 512.329]","(-0.08, 16.0]"
92,0,1,"Chaffee, Mr. Herbert Fuller",male,46.0,1,0,61.1750,S,3,0,Mr,"(31.0, 512.329]","(32.0, 48.0]"
314,0,2,"Hart, Mr. Benjamin",male,43.0,1,1,26.2500,S,3,0,Mr,"(14.454, 31.0]","(32.0, 48.0]"
701,1,1,"Silverthorne, Mr. Spencer Victor",male,35.0,0,0,26.2875,S,1,1,Mr,"(14.454, 31.0]","(32.0, 48.0]"
594,0,2,"Chapman, Mr. John Henry",male,37.0,1,0,26.0000,S,3,0,Mr,"(14.454, 31.0]","(32.0, 48.0]"


In [22]:
data_val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 16 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            418 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           418 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
Familysize     418 non-null int64
IsAlone        418 non-null int64
Title          418 non-null object
Farebin        418 non-null category
Agebin         418 non-null category
dtypes: category(2), float64(2), int64(6), object(6)
memory usage: 46.8+ KB


In [23]:
# sklearnとpandasを使って、カテゴリカルをダミーに
print(data_cleaner)

[     Survived  Pclass                                               Name  \
0           0       3                            Braund, Mr. Owen Harris   
1           1       1  Cumings, Mrs. John Bradley (Florence Briggs Th...   
2           1       3                             Heikkinen, Miss. Laina   
3           1       1       Futrelle, Mrs. Jacques Heath (Lily May Peel)   
4           0       3                           Allen, Mr. William Henry   
5           0       3                                   Moran, Mr. James   
6           0       1                            McCarthy, Mr. Timothy J   
7           0       3                     Palsson, Master. Gosta Leonard   
8           1       3  Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)   
9           1       2                Nasser, Mrs. Nicholas (Adele Achem)   
10          1       3                    Sandstrom, Miss. Marguerite Rut   
11          1       1                           Bonnell, Miss. Elizabeth   
12         

In [24]:
# sex、embarked、title(object)と、farebin、Agebin(category)を変換
label = LabelEncoder()
data1.info()
data_val.info()
data1.sample(10)
# 全ての種類のカテゴリカル変数に数字をふってそれを割り振る
# onehotにすると、列に分けて列番号が
# https://qiita.com/kibinag0/items/723f95277263921650b4
# fitでインスタンスを作って(カテゴリカル全てを格納してる？)、transformすると、encodeする。
for dataset in data_cleaner:    
    dataset['Sex_Code'] = label.fit_transform(dataset['Sex'])
    dataset['Embarked_Code'] = label.fit_transform(dataset['Embarked'])
    dataset['Title_Code'] = label.fit_transform(dataset['Title'])
    dataset['Agebin_Code'] = label.fit_transform(dataset['Agebin'])
    dataset['Farebin_Code'] = label.fit_transform(dataset['Farebin'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
Survived      891 non-null int64
Pclass        891 non-null int64
Name          891 non-null object
Sex           891 non-null object
Age           891 non-null float64
SibSp         891 non-null int64
Parch         891 non-null int64
Fare          891 non-null float64
Embarked      891 non-null object
Familysize    891 non-null int64
IsAlone       891 non-null int64
Title         891 non-null object
Farebin       891 non-null category
Agebin        891 non-null category
dtypes: category(2), float64(2), int64(6), object(4)
memory usage: 85.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 16 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            418 non-null float64
SibSp          418 non-null int64
Parch          418 non-null in

In [25]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 19 columns):
Survived         891 non-null int64
Pclass           891 non-null int64
Name             891 non-null object
Sex              891 non-null object
Age              891 non-null float64
SibSp            891 non-null int64
Parch            891 non-null int64
Fare             891 non-null float64
Embarked         891 non-null object
Familysize       891 non-null int64
IsAlone          891 non-null int64
Title            891 non-null object
Farebin          891 non-null category
Agebin           891 non-null category
Sex_Code         891 non-null int64
Embarked_Code    891 non-null int64
Title_Code       891 non-null int64
Agebin_Code      891 non-null int64
Farebin_Code     891 non-null int64
dtypes: category(2), float64(2), int64(11), object(4)
memory usage: 120.3+ KB


In [26]:
#define y variable aka target/outcome
Target = ['Survived']

#define x variables for original features aka feature selection
data1_x = ['Sex','Pclass', 'Embarked', 'Title','SibSp', 'Parch', 'Age', 'Fare', 'FamilySize', 'IsAlone'] #pretty name/values for charts
data1_x_calc = ['Sex_Code','Pclass', 'Embarked_Code', 'Title_Code','SibSp', 'Parch', 'Age', 'Fare'] #coded for algorithm calculation

In [27]:
data1_xy =  Target + data1_x
print('Original X Y: ', data1_xy, '\n')

Original X Y:  ['Survived', 'Sex', 'Pclass', 'Embarked', 'Title', 'SibSp', 'Parch', 'Age', 'Fare', 'FamilySize', 'IsAlone'] 



In [28]:
#define x variables for original w/bin features to remove continuous variables
data1_x_bin = ['Sex_Code','Pclass', 'Embarked_Code', 'Title_Code', 'FamilySize', 'AgeBin_Code', 'FareBin_Code']
data1_xy_bin = Target + data1_x_bin
print('Bin X Y: ', data1_xy_bin, '\n')

Bin X Y:  ['Survived', 'Sex_Code', 'Pclass', 'Embarked_Code', 'Title_Code', 'FamilySize', 'AgeBin_Code', 'FareBin_Code'] 



In [29]:
#define x and y variables for dummy features original
data1_dummy = pd.get_dummies(data1[data1_x])

KeyError: "['FamilySize'] not in index"